**Author:**  M. Adil Fayyaz


**Email:** adilfayyaz6@gmail.com


In [ ]:
# Import Libraries
import nltk
from nltk.probability import ConditionalFreqDist
import copy
import numpy as np
import pandas as pd

# File Reading

Read the data.txt which will be used as our Vocabulary 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
with open('/content/drive/MyDrive/data.txt','r') as f:
  lines = f.readlines()
data = ""
# Append the lines into a string
for i in lines:
    data+=i
data=data.split('\n')


Mounted at /content/drive


Read the misspellings.txt 

In [ ]:
with open('/content/drive/MyDrive/misspellings.txt','r') as f:
  lines = f.readlines()
misspelled = ""
for i in lines:
    misspelled+=i
misspelled=misspelled.split('\n')
misspelled
# Create a dictionary which will store the correct word as key and a list of 
# wrong words as the value
misspellings = {}
firstRow = 1
for row in misspelled:
  if firstRow:
    firstRow = 0
  else:
    # Slice the string read according to commas and tabs
    correct_word = row.split(',')[0]
    wrong_words = row.split(',')[-1]
    wrong_words = wrong_words[1:]
    wrong_words_list = wrong_words.split('\t')
    misspellings[correct_word] = wrong_words_list


# Language Model

Create a unigrams list. This list contains the words existing in the vocabulary

In [ ]:
unigrams = list()
for i in range(len(data)):
  words = data[i].split()
  for j in range(len(words)):
    w1 = words[j]
    unigrams.append(w1)

Calculate the actual value of unigrams, i.e. count the number of occurences that a particular word occured in the vocabulary. Store this in a dictionary, where the key represents the word and the value represents the count.

Store the Unique words, i.e. the keys list in a separate variable

In [ ]:
unigram_word_dictionary = {}
for w in unigrams:
  if unigram_word_dictionary.get(w):
    unigram_word_dictionary[w] += 1
  else:
    unigram_word_dictionary[w] = 1
global UniqueWords
UniqueWords = unigram_word_dictionary.keys()

Calculate the count unigram used to count the characters existing in the vocabulary ( Needed to calculate P(x|w) ). Store the unigram in a dictionary, where the key represents an alphabet and the value represents the number of times that character appeared in the vocabulary.

In [ ]:
# Make Unigrams for the Characters
count_unigram_prob = {} # Empty dictionary 
for word in unigrams:
  for alphabet in word:
    if alphabet in count_unigram_prob: # If alphabet already exists in the dict
      count_unigram_prob[alphabet] += 1
    else: # If alphabet read for the first time
      count_unigram_prob[alphabet] = 1


Calculate the count bigram used to count the characters that appear consecutively in the vocabulary ( Needed to calculate P(x|w) ). Store the bigram in a dictionary, where the key represents a tuple of two alphabets and the value represents the number of times the two characters appeared consecutively in the vocabulary.

In [ ]:
# Make Bigrams for the Characters
count_bigram = {} # Empty Dictionary
for correctWord in unigrams:
  correctWord = '#' + correctWord
  for x in range(len(correctWord) - 1): 
    w1 = correctWord[x]
    w2 = correctWord[x+1]
    if tuple((w1,w2)) in count_bigram: # If tuple already exists in the dict
      count_bigram[tuple((w1,w2))] += 1
    else: # If the tuple is read for the first time
      count_bigram[tuple((w1,w2))] = 1 

Function that calculates the probability of a word. Uses the unigram word dictionary, used above to train the model, to return the probability of a word appearing given a vocabulary. i.e. P(w) -> prior

In [ ]:
# Language Model
# Train the Unigram model -> P(w)

def getUnigramProbabilities(word): 
  if unigram_word_dictionary.get(word):
    return unigram_word_dictionary[word]/float(len(unigrams))  # Get word from dict, divide by number of words in vocab
  else:
    return -1


# Error Model

Function getEditOperation takes two parameters. w represents the correct word and x represents the typed word. Returns the operation and x, y coordinates to be added in the confusion matrix 

---

**Logic:** Compare the length between the two strings. According to the length find out if the operation was an insert or delete operation (Where the difference between the strings will vary by length 1). If the length of the strings is equal, then the words either have a transpose or substitute operation. To calculate the transpose operation between two strings sort both the strings and compare them, if they are equal and the index of the misplaced word is varied by length 1 (To ensure 1 edit dist) then the operation is transpose, otherwise it is a substitution operation

In [ ]:
def getEditOperation(w,x):
  # If both words are different
  if w != x:
    w_list = list(w)
    x_list = list(x)
    # INSERT
    if len(w) <= len(x) - 1:
      foundInsert = 0
      for j in range(len(x_list) - 1):
        if x_list[j] != w_list[j] and j == 0:
          break
        if x_list[j] != w_list[j]:
          foundInsert = 1
          # The last letter of w is x and the current letter is y
          x = w_list[j-1]
          y = x_list[j]
          break
      if foundInsert:    
        return "insert", x, y
      else:
        # Insert operation at the first index
        if x_list[0] != w_list[0]:
          return "insert", '#', x_list[0]
        # Insert operation at the last index
        else:
          return "insert", w_list[-1], x_list[-1]

    # DELETE
    elif len(w) >= len(x) + 1:
      foundDelete = 0
      for i in range(len(w_list) - 1):
        if x_list[i] != w_list[i] and i==0:
          break
        if x_list[i] != w_list[i]:
          foundDelete = 1
          x = w_list[i-1]
          y = w_list[i]
          break
      if foundDelete:
        
        return "delete", x, y
      else:
        # Delete Operation at the first index
        if x_list[0] != w_list[0]:
          return "delete", '#', w_list[0]
        else:
          return 'delete', w_list[-2], w_list[-1]

    elif len(w) == len(x): # Either substitute or transpose
      copyW = copy.deepcopy(w)
      copyX = copy.deepcopy(x)
      # SUBSTITUTE
      if sorted(copyW) != sorted(copyX):  
        for i in range(len(w_list)):
            if w_list[i] != x_list[i]:
              return "substitute", x_list[i], w_list[i]
      # TRANSPOSE
      else:
        istrans = 0
        for a in range(len(x)-1):
          if x[a] == w[a+1] and x[a+1] == w[a]: # Check if indexes swapped are consecutive
            istrans = 1
            break
        if istrans:
          for i in range(len(w_list)-1):
            if w_list[i] != x_list[i]:
              return "transpose", w_list[i], w_list[i+1] 
  else:
    return "same", -1, -1

Create the Confusion Matrix for all the Insert, Delete, Transpose and Substitute Tables. Use the misspellings to populate the DataFrames using the getEditOperation function defined above

In [ ]:
# Error Model -> P(x|w)
# Character level insert, delete, transpose and substitue tables

# X and Y labels for the DataFrames
x_values = ['#','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
y_values = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
# Create dataframes for the confusion matrices
# Insert DF
InsertDF = pd.DataFrame(columns = y_values, index = x_values)
InsertDF.fillna(0, inplace = True) 

# Delete DF
DeleteDF = pd.DataFrame(columns = y_values, index = x_values)
DeleteDF.fillna(0, inplace = True) 

# Substitue DF
SubstituteDF = pd.DataFrame(columns = y_values, index = x_values)
SubstituteDF.fillna(0, inplace = True) 

#Transpose DF
TransposeDF = pd.DataFrame(columns = y_values, index = x_values)
TransposeDF.fillna(0, inplace = True) 


for correctWord in misspellings:
  # Populate the confusion matrices
  for incorrectWord in misspellings[correctWord]:
    editOperation, x, y = getEditOperation(correctWord,incorrectWord)
    if editOperation == 'insert':
      InsertDF[y][x] += 1
    elif editOperation == 'delete':
      DeleteDF[y][x] += 1
    elif editOperation == 'substitute':
      SubstituteDF[y][x] += 1
    elif editOperation == 'transpose':
      TransposeDF[y][x] += 1

print(InsertDF)
print("*******")
print(DeleteDF)
print("*******")
print(SubstituteDF)
print("*******")
print(TransposeDF)


     a    b    c    d    e    f    g  ...    t    u    v    w    x    y    z
#  594  650  705  682  710  707  683  ...  718  730  732  710  771  681  744
a  548  693  644  652  662  641  656  ...  620  670  633  626  656  634  648
b   57   81   95   97   60   86   66  ...   96   98   90  100   97   81   88
c   55   64   69   66   58   67   68  ...   76   60   61   59   79   69   55
d   99  109  127  136   97  148  114  ...  125  126  131  115  130  131  149
e  273  298  248  230  235  272  270  ...  236  277  286  259  243  241  270
f   23   39   58   38   32   44   38  ...   40   34   43   39   28   39   31
g   56   88   88   80   74   74   74  ...   85   72   70   87   65   84   81
h  112  206  210  237  185  228  234  ...  219  222  222  232  215  224  218
i  229  254  259  252  250  246  248  ...  240  276  275  278  242  266  254
j   38   53   53   42   50   50   49  ...   40   33   54   55   39   48   61
k   88  123  119  136  113  110  137  ...  112  104  112  112  127   93  119

Returns the value against the tuple i, j in the count bigram defined above

In [ ]:
def getCountBigram(i, j):
  # Return the value against the tuple i, j in the 
  # count bigram
  return (count_bigram[i,j])

Returns the number of character counts in the unigram calculated above 

In [ ]:
# Count of individual alphabets in correct misspellings dictionary
def getCountOfWi(character):
  # If the character is '#' , we know that # technically is the starting character
  # of every word, so safe to assume that the number of times '#' will appear in the
  # vocabulary will be equal to the number of words.
  # Note - here unigrams represents individual words list
  if character == '#':
    return len(unigrams)
  
  return count_unigram_prob[character] 

Function getProbabilityOfWord takes the correct word and the typed word as parameter. Computes the value of P(x|w) by fetching the edit operation performed between both the words and then returning the probability according to the individual formulas of Insert, Delete, Substitute, and Transpose

In [ ]:
def getProbabilityOfWord(w,x):
  operation, i, j = getEditOperation(w,x)
  if operation == 'insert':
    probability = InsertDF[j][i]/getCountOfWi(i)
  elif operation == 'delete':
    probability = DeleteDF[j][i]/getCountBigram(i,j)
  elif operation == 'substitute':
    probability = SubstituteDF[j][i]/getCountOfWi(j)
  elif operation == 'transpose':
    probability = TransposeDF[j][i]/getCountBigram(i,j)
  elif operation == 'same':
    probability = 1
  return probability



# Candidate Words Generation

Function isTranspose returns a bool if the passed strings w and x have a transpose operation between them

In [ ]:
# Function checks if two words have a transpose operation between them
def isTranspose(w,x):
  if sorted(w) == sorted(x):
    for a in range(len(x)-1):
      if x[a] == w[a+1] and x[a+1] == w[a]: # Checks if the alphabets were consecutive
        return True
  return False

Function getEditDistance calculates the edit distance given two strings w and x using the levenshtein distance formula - to measure the difference between two sequences

In [ ]:
# Calculate the edit distance between the given and candidate word
def getEditDistance(w,x):
  len_w = len(w)
  len_x = len(x)
  edits = [[0 for e in range(len_x + 1)] for u in range(len_w + 1)]
  for i in range(len(w) + 1):
    for j in range(len(x) + 1):
      if i == 0:
        edits[i][j] = j
      elif j == 0:
        edits[i][j] = i
      elif w[i-1] == x[j-1]:
        edits[i][j] = edits[i-1][j-1]
      else:
        edits[i][j] = 1 + min(edits[i-1][j], edits[i][j-1], edits[i-1][j-1])
  return edits[len_w][len_x]


Function getCandidateWords, returns the candidates 1 edit distance away from the typed word - x. Iterates over the set of unique words in our vocabulary and calculates the edit distance between them and the typed word. Adds them to a list of candidate words if the edit distance is equal to 1.

Note: The function returns a single word if the typed word x already exists in the vocabulary. In this case, the same word is returned. I.e. Edit distance calculated between both the words(same) will be equal to zero (0)

In [ ]:
# Given a word x, return a list of candidate words from the Vocabulary
def getCandidateWords(x):
  candidateWords = []
  zeroEditCandidate = []
  if x in UniqueWords:
    print("*** Word exists in the Vocabulary ***")
    zeroEditCandidate.append(x)
    return zeroEditCandidate
  
  for correctWord in UniqueWords:
    editDist = getEditDistance(correctWord, x)
    # getEditDistance returns insert, delete and subs only. However a transpose
    # operation is considered as a substitution with edit Dist = 2. 
    # Hence, check if the words are transpose then subtract 1 from the edit dist
    # to find the edit distance for the transpose operation
    isTrans = isTranspose(correctWord, x)
    if isTrans:
      editDist -= 1
    if editDist == 1:
      candidateWords.append(correctWord)

  availableCandidates = []
  # Precautionary check in case the entire data is not read
  for w in candidateWords:
    try:
      p = getUnigramProbabilities(w)
      if p > 0:
        availableCandidates.append(w)
    except: # did not find the word in Vocab
      xyz=1 # garbage assignment

  return availableCandidates



# Selection Model Using ArgMax

Function getProbableWord takes as parameter the typed word and returns the word with the highest probability. The probability P(x|w) is fetched from getProbabilityOfWord function and the probability P(w) is fetched from getUnigramProbabilities function. The product of both the probabilities is stores in an options dictionary with the key as a candidate word and the value as the probability.

If the program finds a list of options it returns the ArgMax = Max of the list according to probability. Else, if the program does not find a candidate word, it returns None.

In [ ]:
def getProbableWord(x):
  candidates = getCandidateWords(x)
  # print(candidates)
  options = {} # Dictionary
  for w in candidates:
    prob_xw = getProbabilityOfWord(w,x)  # P(x|w)
    prob_w = getUnigramProbabilities(w)  # P(w)
    result = prob_xw * prob_w            # P(x|w) . P(w)
    options[w] = result                  # Store the result in the dict
    print("Candidate: ", w, " Probability: ", result)
  if options:
    likely_word = max(options, key=options.get)  # Likely word is the word with the maximum probability 
    return likely_word
  else: # If no options/candidates found
    print("*** No Candidates Found ***")
    return "None"


# Testing/Output

In [ ]:
# Testing 
x = 'mujhel'
likely = getProbableWord(x)
print("Suggested Word is: ", likely)

Candidate:  mujhe  Probability:  7.450380326318264e-08
Candidate:  mujhey  Probability:  2.231249309368189e-11
Suggested Word is:  mujhe


In [ ]:
x = 'kaesa'
likely = getProbableWord(x)
print("Suggested Word is: ",likely)

Candidate:  aesa  Probability:  1.0370924043010484e-10
Candidate:  kaisa  Probability:  8.487599927204714e-09
Candidate:  kesa  Probability:  2.0943405647538557e-11
Candidate:  kasa  Probability:  1.8482692760567328e-11
Candidate:  kaasa  Probability:  3.58351637919582e-11
Candidate:  khesa  Probability:  2.157150194830464e-11
Suggested Word is:  kaisa


In [ ]:
x = 'adile'
likely = getProbableWord(x)
print("Suggested Word is: ",likely)

Candidate:  adil  Probability:  4.0370865485362967e-10
Candidate:  adle  Probability:  4.367925309691912e-12
Candidate:  adine  Probability:  2.265205203361891e-12
Candidate:  dile  Probability:  1.4875709653115589e-12
Candidate:  adele  Probability:  1.220692398292538e-10
Suggested Word is:  adil


In [ ]:
# Word already exists in the Vocabulary, return the same word
x = 'nahi'
likely = getProbableWord(x)
print("Suggested Word is: ",likely)

*** Word exists in the Vocabulary ***
Candidate:  nahi  Probability:  0.012080763051931738
Suggested Word is:  nahi


In [ ]:
# Word does not return any candidate words
# Returns 'None' i.e. no word found
x = 'nahasdksjai'
likely = getProbableWord(x)
print("Suggested Word is: ",likely)

*** No Candidates Found ***
Suggested Word is:  None


In [ ]:
x = 'kitaa'
likely = getProbableWord(x)
print("Suggested Word is: ",likely)

Candidate:  kiaa  Probability:  7.180690348198157e-12
Candidate:  kitna  Probability:  1.2963851350078707e-09
Candidate:  kitab  Probability:  6.765369612314775e-11
Candidate:  kita  Probability:  1.969150264291617e-11
Candidate:  kitaab  Probability:  1.5484561487365123e-09
Candidate:  kiyaa  Probability:  7.921392271476204e-12
Candidate:  kitta  Probability:  5.590334732247482e-12
Candidate:  kitana  Probability:  1.8237941720142664e-12
Suggested Word is:  kitaab


In [ ]:
# Word does not return any candidate words
# Returns 'None' i.e. no word found
x = 'karnaed'
likely = getProbableWord(x)
print("Suggested Word is: ",likely)

*** No Candidates Found ***
Suggested Word is:  None


In [ ]:
x = 'usary'
likely = getProbableWord(x)
print("Suggested Word is: ",likely)

Candidate:  usay  Probability:  9.18408663826041e-09
Candidate:  sary  Probability:  2.1389467364858627e-11
Candidate:  umary  Probability:  2.9070395480200804e-12
Candidate:  usar  Probability:  9.914359709551538e-13
Suggested Word is:  usay
